# Atomic

## arch/x86/include/asm/barrier.h

```c
/*
 * Some non-Intel clones support out of order store. wmb() ceases to be a
 * nop for these.
 */
#define mb() alternative("lock; addl $0,0(%%esp)", "mfence", X86_FEATURE_XMM2)
#define rmb() alternative("lock; addl $0,0(%%esp)", "lfence", X86_FEATURE_XMM2)
#define wmb() alternative("lock; addl $0,0(%%esp)", "sfence", X86_FEATURE_XMM)
#else
#define mb() 	asm volatile("mfence":::"memory")
#define rmb()	asm volatile("lfence":::"memory")
#define wmb()	asm volatile("sfence" ::: "memory")
#endif
```

1. If cpu doesn't support `mfence,lfence,sfence`, use `lock` instruction. The LOCK prefix ensures that the CPU has exclusive ownership of the appropriate cache line for the duration of the operation, and provides certain additional ordering guarantees. This may be achieved by asserting a bus lock, but the CPU will avoid this where possible. 

2. [Lock & memory order](https://stackoverflow.com/questions/60332591/why-is-lock-a-full-barrier-on-x86)

3. [缓存一致性MESI](https://cloud.tencent.com/developer/article/1548942)